# Stroke Prediction Using Deep Learning

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Loading in the data and getting a simple overview of it

In [121]:
data = pd.read_csv('C:\\Users\\Tyron\\OneDrive\\Desktop\\Machine Learning Projects\\Machine-Learning-Projects\\Stroke_Prediction\\healthcare-dataset-stroke-data.csv')
data.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [123]:
data.isna().any()

id                   False
gender               False
age                  False
hypertension         False
heart_disease        False
ever_married         False
work_type            False
Residence_type       False
avg_glucose_level    False
bmi                   True
smoking_status       False
stroke               False
dtype: bool

In [124]:
data.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

### Data Preprocessing

#### Splitting the data into train and test sets

In [125]:
data = data[data['gender'] != 'Other']
stroke_data = data[data['stroke'] == 1].sample(249, random_state=42)
no_stroke_data = data[data['stroke'] == 0].sample(249, random_state=42)

In [143]:
test = pd.read_csv('test.csv')
submission = test['id']

In [144]:
test = test[test['work_type'] != 'Never_worked']

In [145]:
test = test[test['gender'] != 'Other']

In [146]:
test = test.drop(['id'], axis=1)

In [130]:
equal_sample = pd.concat([stroke_data.reset_index(), no_stroke_data.reset_index()], axis=0)
X = equal_sample.drop(['index', 'id', 'stroke'], axis=1)
y = equal_sample['stroke']

### Creating a Pipeline to automate data preprocessing

In [131]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

In [132]:
class Stroke_Pipeline():
    def __init__(self, data):
        self.data = data
        self.num_pipeline = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())
        self.cat_pipeline = make_pipeline(OneHotEncoder())
    
    def transform(self):
        num_attrib = self.data.select_dtypes(['int64', 'float64']).columns
        cat_attrib = self.data.select_dtypes('object').columns

        pipe = ColumnTransformer([
            ('num', self.num_pipeline, num_attrib),
            ('cat', self.cat_pipeline, cat_attrib)
        ])

        piped_data = pipe.fit_transform(self.data)
        return tf.convert_to_tensor(piped_data)

In [133]:
X = Stroke_Pipeline(X).transform()
y = tf.convert_to_tensor(y)

In [134]:
X

<tf.Tensor: shape=(498, 19), dtype=float64, numpy=
array([[ 0.94631846, -0.45687839, -0.36660108, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.85486579,  2.18876624,  2.72776061, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.12324441, -0.45687839,  2.72776061, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.05966093, -0.45687839, -0.36660108, ...,  0.        ,
         0.        ,  1.        ],
       [-0.83700864, -0.45687839, -0.36660108, ...,  0.        ,
         1.        ,  0.        ],
       [-1.93444071, -0.45687839, -0.36660108, ...,  0.        ,
         0.        ,  0.        ]])>

In [135]:
y.shape

TensorShape([498])

### Model Creation

In [75]:
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(19,)),
    tf.keras.layers.Dense(498, activation=leaky_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(498, activation=leaky_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(249, activation=leaky_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(249, activation=leaky_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(125, activation=leaky_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [76]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'],
    loss='binary_crossentropy'
)

In [80]:
model.fit(X, y, epochs=100)

Epoch 1/100
16/16 [==============================] - 0s 6ms/step - loss: 0.2247 - accuracy: 0.8956
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 0.1877 - accuracy: 0.9157
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.1731 - accuracy: 0.9177
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 0.1855 - accuracy: 0.9096
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.1675 - accuracy: 0.9237
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.1954 - accuracy: 0.8956
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.1906 - accuracy: 0.9197
Epoch 8/100
16/16 [==============================] - 0s 8ms/step - loss: 0.1389 - accuracy: 0.9418
Epoch 9/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2019 - accuracy: 0.9076
Epoch 10/100
16/16 [==============================] - 0s 7ms/step - loss: 0.2043 - accuracy: 0.8976
Epoch 11/

In [136]:
test = Stroke_Pipeline(test).transform()

In [140]:
y_pred = model.predict(test)

319/319 [==============================] - 1s 2ms/step


###  Model Evaluation

#### Evaluation on Training Data

### Saving the model